In [183]:
from functools import partial

from common.conics import plot_conics, ConicProjector, ellipse_axes, scale_det, conic_center
from common.coordinates import ENU_system
from src.matching import CraterDatabase, CoplanarInvariants
from src.common.data import DataGenerator
from src.detection.metrics import gaussian_angle_distance
import src.common.constants as const
import numpy as np
import numpy.linalg as LA
from sklearn.linear_model import RANSACRegressor, TheilSenRegressor
from src.matching.position_estimation import derive_position_ransac
from scipy.stats import median_absolute_deviation
from scipy.optimize import least_squares, minimize
from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist


import matplotlib.pyplot as plt

In [184]:
db = CraterDatabase.from_file("../data/lunar_crater_database_robbins_2018.csv",
                              latlims=(0, 20),
                              longlims=(0, 20),
                              diamlims=(7, 40)
                              )
gen = DataGenerator.from_robbins_dataset(file_path="../data/lunar_crater_database_robbins_2018.csv", diamlims=(7, 40), axis_threshold=(1, 250))
gen.set_coordinates(10, 10, 350, convert_to_radians=True)
gen.position

Surrender Python Client
code revision : c17439eea9ae041bf49364d67a2a232fc89c1faa


array([[2024.16623442],
       [ 356.91512057],
       [ 362.42111161]])

In [204]:
C_cat, r_cat = gen.visible_catalogue_craters()
A_detections = gen.project_crater_conics(C_cat, r_cat)
crater_triads, key = next(CoplanarInvariants.match_generator(
    A_craters=A_detections,
    max_iter=1,
    batch_size=100
))

top_n = 5
crater_triads = np.repeat(crater_triads[:, None, :], top_n, axis=1)
# A_db_ = np.repeat(A_detections[crater_triads][:, None, ...], top_n, axis=1).reshape(-1, 3, 3)

match_idxs, _ = db.query(key, k=top_n, return_distance=True)

dist = np.abs((key[:, None, :] - db._features[match_idxs]) / key[:, None, :]).mean(-1)
dist_filter = dist < 0.1
r_query, C_query = db[match_idxs[dist_filter]]
A_query = A_detections[crater_triads[dist_filter]]
A_query, r_query, C_query = A_query.reshape(-1, 3, 3), r_query.reshape(-1, 3, 1), C_query.reshape(-1, 3, 3)
A_query.shape, r_query.shape, C_query.shape

((84, 3, 3), (84, 3, 1), (84, 3, 3))

In [202]:
k = np.array([0., 0., 1.])[:, None]
S = np.concatenate((np.identity(2), np.zeros((1, 2))), axis=0)

T = gen.T
K = gen.K

B_craters = T @ K.T @ A_query @ K @ LA.inv(T)
T_EM = np.concatenate(ENU_system(r_query), axis=-1)
T_ME = LA.inv(T_EM)

A_ = S.T @ T_ME @ B_craters
A = A_.reshape(-1, 3)
b_ = S.T @ T_ME @ B_craters @ r_query
b = b_.reshape(-1, 1)

median_absolute_deviation

<function numpy.lib.utils._Deprecate.__call__.<locals>.newfunc(*args, **kwds)>

In [203]:
degraded_position = gen.position + np.random.randn(3, 1)*100

min_inliers = int(len(A_query)/top_n/8)

ransac = RANSACRegressor(residual_threshold=1000, max_trials=10000)
ransac.fit(A, b)

theil_sen = TheilSenRegressor(n_jobs=-1, max_iter=1000, tol=1e-4, fit_intercept=False, n_subsamples=50)
theil_sen.fit(A, b)

inlier_mask = np.logical_and.reduce(ransac.inlier_mask_.reshape(b_.shape), axis=(-1, -2))
num_inliers = inlier_mask.sum()

est_pos_lsq, _, _, _ = LA.lstsq(A, b)

est_pos_ransac = ransac.estimator_.coef_.T
est_pos_theil_sen = theil_sen.coef_.T
print(f"Least squares:\tPosition error: {LA.norm(est_pos_lsq - gen.position):.2f} km")
print(f"RANSAC:\t\tPosition error: {LA.norm(est_pos_ransac - gen.position):.2f} km | Inliers: {num_inliers}")
print(f"Theil Sen:\tPosition error: {LA.norm(est_pos_theil_sen - gen.position):.2f} km")

Least squares:	Position error: 298.21 km
RANSAC:		Position error: 0.00 km | Inliers: 65
Theil Sen:	Position error: 3150.95 km


C:\Users\w_dop\miniconda3\envs\crater-detection\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\w_dop\miniconda3\envs\crater-detection\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [10]:
C_inlier, r_inlier = C_query.reshape(-1, 3, 3)[inlier_mask], r_query.reshape(-1, 3, 1)[inlier_mask]

projector = ConicProjector(position=est_pos, attitude=gen.T)

A_projected = projector.project_crater_conics(C_inlier, r_inlier)
A_matched = A_db[inlier_mask]

NameError: name 'est_pos' is not defined

In [ ]:
sigma_pix = 3
primary_body_radius = const.RMOON
max_alt = 500

divergence = gaussian_angle_distance(A_projected, A_matched)

a_i, b_i = ellipse_axes(A_projected)

sigma = (0.85 / np.sqrt(a_i * b_i)) * sigma_pix

reprojection_mask = ((divergence / sigma) ** 2) <= 13.276
gen.position - derive_position_ransac(A_craters=A_matched[reprojection_mask], r_craters=r_inlier[reprojection_mask], T=gen.T, K=gen.K, min_inliers=10)

In [ ]:
est_pos2, num_matches = db.query_position(A_detections, T=T, K=K, max_trials=1000)
est_pos2 - gen.position

Surrender Python Client
code revision : c17439eea9ae041bf49364d67a2a232fc89c1faa


Exception ignored in: <function surrender_client_base.__del__ at 0x0000017D8563ECA8>
Traceback (most recent call last):
  File "C:\Users\w_dop\miniconda3\envs\crater-detection\lib\site-packages\surrender\surrender_client_base.py", line 59, in __del__
    if self.isConnected():
  File "C:\Users\w_dop\miniconda3\envs\crater-detection\lib\site-packages\surrender\surrender_client_base.py", line 240, in isConnected
    return self._stream and self._sock and self._sock.isOpen();
RuntimeError: wrapped C/C++ object of type QTcpSocket has been deleted


array([[2024.16623442],
       [ 356.91512057],
       [ 362.42111161]])